<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name: Zaki Kidane


<br>




**Some things that might be useful**


In [2]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
import math
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [33]:
#the instantiation for both classes as well as the random_player function
#were used from the class notebooks
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = moves

class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        global num_nodes
        num_nodes = 0 #every time new game starts number of nodes 
                    #explored is reset to 0
        global num_nodes_minmax
        num_nodes_minmax = 0 #every time new game starts number of nodes 
                    #explored is reset to 0
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
#                     self.display()
                    return self.state.utility   

<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [4]:
def random_player(game):
    '''A player that chooses a legal move at random.'''
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]


<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

The long term win seems to be about 32-33%.

In [37]:
#to do this we simply change the dimensions of TicTacToe class to 4,4,4 and 
#run the same loop as we did in class notebooks for a 1000 iterations
n_iter = 1000
wins = 0
draws = 0
losses = 0
for k in range(n_iter):
    ttt= TicTacToe(4,4,4)
    out = ttt.play_game(random_player, random_player)
    print('')
    print('----------')
    print('')
    if out== 1:
        wins += 1
    elif out == -1:
        losses += 1
    else:
        draws += 1

        


cell works
O O O O 
X O O X 
X X O X 
O X X X 

----------

O X X X 
O X . X 
O . X O 
O O . . 

----------

X O X O 
O X X X 
O X O X 
O O O X 

----------

. X . X 
O X X O 
O X X O 
O X O . 

----------

O X O X 
O O X O 
O O X X 
X O X X 

----------

O X O X 
X O X X 
O O X X 
X O O O 

----------

O X . . 
. X O O 
. X . X 
. X . O 

----------

O X O X 
O X O X 
X X X O 
O O X O 

----------

X X . X 
O X O O 
O O O O 
X . X X 

----------

O X O O 
X X O O 
X O X X 
X O O X 

----------

O X X O 
O O X X 
X O X O 
O X O X 

----------

X . . X 
O . . X 
O O . X 
. O . X 

----------

X O O X 
O O X X 
X O . X 
O X O X 

----------

O X . X 
O X O O 
O X O X 
X X X O 

----------

O O X X 
X O X . 
. O X X 
X O O O 

----------

X X O X 
X O O O 
X X O X 
O O X O 

----------

X O X X 
X O O O 
O O X X 
O O X X 

----------

X O O X 
X O X X 
O O X O 
O X O X 

----------

O O . O 
X X X X 
X O O X 
O O X X 

----------

X O X O 
X O X O 
X X O X 
O O X O 

----------

X X X O 


O X O O 
X O X X 
X X X O 
O O O X 

----------

X O . . 
X X O O 
X O O X 
X O X . 

----------

X X X O 
O O O X 
X X O X 
O O O X 

----------

. O X X 
O O X X 
O X O O 
X O X X 

----------

X O X O 
X O X O 
O O X X 
X X O O 

----------

O X X O 
O X O X 
X O O X 
O X X O 

----------

X X O X 
O O O X 
X X X O 
O O X O 

----------

X X X O 
O X X O 
O O O O 
O X X X 

----------

X . O O 
X O O O 
X X . X 
X . X O 

----------

O . O X 
X O . X 
. . O X 
. . X O 

----------

X O . . 
X X X X 
O . O O 
X O X O 

----------

. . . X 
O . X . 
O X . . 
X O . . 

----------

X O O X 
X O O X 
O . O X 
X O X X 

----------

O X X O 
X X O X 
X O O X 
X O O O 

----------

X O O X 
X X X . 
X O . . 
X O O O 

----------

O O X O 
O X X . 
O X . X 
O X X O 

----------

O O . . 
X O O X 
X X O O 
X X X O 

----------

O X . O 
X X X X 
. . O O 
. . . . 

----------

X X O O 
X O O X 
O X O O 
O X X X 

----------

O . O O 
O O X . 
X X X X 
X X . O 

----------

X O O . 
X . . O 
X 

O X O O 
O . X X 
X X X X 
O O O X 

----------

X O O X 
. X O O 
O O X X 
O X X X 

----------

O O X O 
X X X O 
O X O X 
O X O X 

----------

O O X X 
X X X O 
X O O O 
X O X O 

----------

X X X O 
O O O X 
O X O X 
X O X O 

----------

X X O O 
O O X X 
X O O X 
O O X X 

----------

O O O X 
X X O X 
O X O X 
O X X O 

----------

X . X . 
O O . O 
X X X X 
O X O O 

----------

X X X O 
X O X X 
X O O O 
O O X O 

----------

X O X X 
O O X O 
X O O O 
X X X O 

----------

X X O O 
O O X X 
O O X O 
O X X X 

----------

O O O X 
O O X X 
X O O X 
X X X O 

----------

X O O X 
O O X O 
O X X O 
O X X X 

----------

X O O O 
O O X O 
X X X O 
O X X X 

----------

O X O O 
X X X O 
X X O O 
X O O X 

----------

X X O X 
O O O O 
X . X . 
X X O O 

----------

. X . . 
O X O X 
O X X X 
O X O O 

----------

X O X X 
X O X O 
X X O O 
O O X O 

----------

X X O X 
X O O X 
O O O X 
O X X O 

----------

O . . X 
O . X O 
X X O X 
X O X O 

----------

O X O O 
X X O X 
X 

X O O X 
X O X X 
O X O O 
O X X O 

----------

O O X O 
O O O X 
X O X X 
X X O X 

----------

X . X O 
X X O O 
X . O O 
X O X . 

----------

O O O O 
O X O X 
X O X X 
X O X X 

----------

X O X O 
O O O X 
X X O X 
O O X X 

----------

O O O O 
X X X O 
. . X . 
. X O X 

----------

X O O X 
X X O O 
O . X X 
O O X X 

----------

X . O . 
. X O X 
X O O X 
O . O X 

----------

X O X O 
O X O X 
X X X O 
O X O O 

----------

X X O X 
O X X O 
O O X O 
X O X O 

----------

O X O X 
X X X O 
X O X O 
X O O O 

----------

X O O O 
O X O X 
O X O X 
O X X X 

----------

X X O O 
O . O X 
X O O O 
X X X X 

----------

X O O X 
X X O O 
X O X X 
O X O O 

----------

X O O X 
X X X O 
O O O X 
X O O X 

----------

O X O X 
X O O O 
X O X X 
O O X X 

----------

X O O O 
O X X O 
X O X O 
. X X X 

----------

O O X X 
X X X O 
X O O O 
X O X O 

----------

X O O O 
O X O O 
X X X O 
X X X O 

----------

X X O O 
O O X X 
X O O X 
O O X X 

----------

O X X O 
X O O X 
O 

X . . O 
O X . O 
. X X O 
X X O O 

----------

X O X X 
O O X O 
X O X X 
O X O O 

----------

O X O X 
X O O O 
O X O X 
O X X X 

----------

O X X O 
X O O X 
O X X X 
O X O O 

----------

O O X O 
X . X . 
. X X X 
O O X O 

----------

X X X X 
O X O O 
O X X X 
. O O O 

----------

X X O X 
X O O X 
O O O X 
X X O O 

----------

O O X X 
O X X . 
O X O X 
X O X O 

----------

X O X X 
O O O X 
X X X O 
X O O O 

----------

O X X O 
X O X O 
X X O O 
O O X X 

----------

X X O X 
O O X O 
X O O X 
O X O X 

----------

O O X X 
O X O O 
X O X X 
O X O X 

----------

X O O X 
X X O O 
O O O X 
X X X O 

----------

O O X O 
X X O X 
X X X O 
O X O O 

----------

. O X O 
X O X O 
X O O O 
X X X X 

----------

O O O O 
. . X X 
X X O O 
X X O X 

----------

O O X O 
O X X X 
X O O O 
X O X X 

----------

X X O X 
X O O O 
O O X X 
O X O X 

----------

O X X O 
X O O X 
O X O X 
O O X X 

----------

O O X O 
X X X O 
. O X X 
O X X O 

----------

O O X X 
. O X X 
O 

O O X . 
. O X . 
X O O O 
X X X X 

----------

O X X X 
X X O O 
O X O X 
O O O X 

----------

X O X O 
X X O X 
O O X X 
O X O O 

----------

. . X X 
O . X O 
X X O O 
X O O X 

----------

X . X . 
O O X . 
O . X O 
O . X X 

----------

X X X O 
X O O X 
O O O O 
X X X O 

----------

X O X X 
X X O O 
O X O X 
O O O X 

----------

X X X X 
O O . O 
O X O O 
X X . . 

----------

X X O O 
O X O O 
O O X X 
X X . X 

----------

X O O O 
X O O X 
O O X X 
X X O X 

----------

X X O X 
X O O O 
O X X O 
O X X O 

----------

O X X O 
X X O X 
O O X O 
O X X O 

----------

O O X O 
X O X X 
X X O O 
O X O X 

----------

X X X O 
O O O O 
X X X O 
O O X X 

----------

X O O X 
X O X X 
O O O X 
O X X O 

----------

O O X X 
X O X X 
O O O O 
X X O X 

----------

O X X X 
X X O X 
X O O O 
X O O O 

----------

O O . X 
. O X . 
O X X . 
X O X . 

----------

O X X X 
O X O O 
X O X O 
X O O X 

----------

X X O X 
X X X X 
O O X O 
. O O O 

----------

X O . X 
O O O X 
X 

In [38]:
print('First-player winning percentage = {}%'.format(wins/n_iter*100))

First-player winning percentage = 33.300000000000004%


<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [25]:
num_nodes = 0 #global variable to count number of nodes expanded
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    
   
    def min_value(game, state, alpha, beta):
        global num_nodes
        num_nodes+=1 #this increment counter for number of nodes expanded
        if game.game_over:
            return game.utility(state, game.state.to_move)
        value = math.inf
        best_move = None
        for move in state.moves:
            value = min(value,max_value(game, game.result(move, state)))
            if value <= alpha:
                return value
            beta = min(value, beta)
        return value

    def max_value(game, state, alpha, beta):
        global num_nodes
        num_nodes+=1  #this increment counter for number of nodes expanded
        
        if game.game_over:
            return game.utility(state, game.state.to_move)
        value = -math.inf
        best_move = None
        for move in state.moves:
            value = max(value,min_value(game, game.result(move, state)))
            if value >= beta:
                return value
            alpha = max(value, alpha)
        return value
    
    #the following is the body of the alphabeta_function
    #that searches to find the next best move
    alpha = -math.inf
    beta = math.inf
    best_max_value = -math.inf
    best_min_value = math.inf
    
    player = game.state.to_move
    best_action = None
    
   
    
    if player == 'X':
        for current_action in game.state.moves:
            best_value = min_value(game, game.result(current_action, game.state), alpha, beta)

            if best_value > alpha:
                alpha = best_value
                best_action = current_action
    if player == 'O':
        for current_action in game.state.moves:
            best_value = max_value(game, game.result(current_action, game.state), alpha, beta)

            if best_value < beta:
                alpha = best_value
                best_action = current_action
   
    
    return best_action  


Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [27]:

n_iter = 10
wins = 0
draws = 0
losses = 0
for k in range(n_iter):
    ttt= TicTacToe(3,3,3)
    out = ttt.play_game(alphabeta_player, random_player)
    print('')
    if out== 1:
        wins += 1
    elif out == -1:
        losses += 1
    else:
        draws += 1
print('Winning percentage for alpha 1st player = {}%'.format((wins/n_iter)*100))
print('Losing percentage for alpha 1st player = {}%'.format(losses/n_iter*100))
print('Nodes expanded by alphabeta player = ', num_nodes)











Winning percentage for alpha 1st player = 90.0%
Losing percentage for alpha 1st player = 10.0%
Nodes expanded by alphabeta player =  21


In [29]:
n_iter = 10
wins = 0
draws = 0
losses = 0
for k in range(n_iter):
    ttt= TicTacToe(3,3,3)
    out = ttt.play_game(random_player, alphabeta_player)
    print('')
    if out== 1:
        losses += 1
    elif out == -1:
        wins += 1
    else:
        draws += 1
print('Winning percentage for random 1st player = {}%'.format((wins/n_iter)*100))
print('Losing percentage for random 1st player = {}%'.format(losses/n_iter*100))
print('Nodes expanded by alphabeta player for last game = ', num_nodes)











Winning percentage for random 1st player = 50.0%
Losing percentage for random 1st player = 30.0%
Nodes expanded by alphabeta player for last game =  20


<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [30]:
ttt= TicTacToe(3,3,3)
out = ttt.play_game(alphabeta_player, random_player)
print('Number of nodes with pruning: ', num_nodes)

Number of nodes with pruning:  21


In [31]:
#minimax player below
num_nodes_minmax = 0 #global variable to count number of nodes expanded
def minimax_player(game):
    return minimax_search(game)

def minimax_search(game):
    
   
    def min_value(game, state):
        global num_nodes_minmax
        num_nodes_minmax+=1 #this increment counter for number of nodes expanded
        if game.game_over:
            return game.utility(state, game.state.to_move)
        value = math.inf
        best_move = None
        for move in state.moves:
            value = min(value,max_value(game, game.result(move, state)))
        return value

    def max_value(game, state):
        global num_nodes_minmax
        num_nodes_minmax+=1  #this increment counter for number of nodes expanded
        
        if game.game_over:
            return game.utility(state, game.state.to_move)
        value = -math.inf
        best_move = None
        for move in state.moves:
            value = max(value,min_value(game, game.result(move, state)))
            alpha = max(value, alpha)
        return value
    
    
    #the beta and alpha here have a different purpose,
    #which is for comparison to get the optimum value (and a move as a result)
    best_max_value = -math.inf
    best_min_value = math.inf
    
    player = game.state.to_move
    best_action = None
    
   
    
    if player == 'X':
        for current_action in game.state.moves:
            current_value = min_value(game, game.result(current_action, game.state))

            if current_value > best_max_value:
                best_max_value = current_value
                best_action = current_action
    if player == 'O':
        for current_action in game.state.moves:
            current_value = max_value(game, game.result(current_action, game.state))

            if current_value < best_min_value:
                best_min_value = current_value
                best_action = current_action
   
    
    return best_action  



In [32]:
ttt= TicTacToe(3,3,3)
out = ttt.play_game(minimax_player, random_player)
print('Number of nodes without pruning: ', num_nodes_minmax)

Number of nodes without pruning:  25


**I noticed on average that 4 nodes were pruned, with alphabeta going into 20 nodes and minimax 24.** 

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

One suitable function would be 
$$ L(d,x) = (d-x)^2 $$
where $x$ is the optimum time when $awkwardness=0$ and $d$ is the time that one would arrive at a party.

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


![title](2b.jpg)

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

![title](image.jpg)

For the second part of the solution:

![title2](c.jpg)

<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [53]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [107]:
#this function returns the sum of the abs(target[i]-state[i]) for all i between 0 and len(state)
#not here that we want to minimize the score where min.score =0 and max.score = 1
def recipe_success(state):
    
    target = np.array([36, 36, 48, 1, 1, 108, 1, 0.5, 96])
    state = np.asarray(state)
    result = 1/((np.sum(np.absolute(np.subtract(target, state))))+1)
    return result

𝑠𝑡𝑎𝑡𝑒=[30,30,40,1,0,100,0.5,0.5,100]
recipe_success(state)

0.028985507246376812

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [167]:
#the following are some helper functions for readability
def GenerateInitialPopulation():
    initial_population = []
    for i in range(10):#create 10 random states
        new_state = (np.random.randint(217, size=len(target)))/2
        initial_population.append(new_state)
    return initial_population
#this function was written as an alternative to the above function
#as suggested by the Professor on Piazza. It sets all values except one 
#to 0 in the state. Eg. [0, 0, 0, 32, 0, 0, 0, 0, 0]
def SuggestedInitialPopulation():
    initial_population = []
    for i in range(10):
        new_state = np.asarray([0 for i in range(9)])
        print(len(new_state))
        new_state[(i%9)] = (np.random.randint(217))/2
        initial_population.append(new_state)
    return initial_population

#this calculate p of reproduction by simply using p = recipe_succes of state / total_p
def CalculateProbability(population):
    list_of_p = []
    total_p = 0 #this is the denominator when calculating p of reproduction
    for state in population:
        total_p+=recipe_success(state)
    for state in population:
        list_of_p.append((recipe_success(state)/total_p))
    return list_of_p

#function was referenced from class notebooks
def Reproduce(parent1, parent2):
    # last DNA snippet from parent1:
    split = np.random.randint(low=1, high=len(target))
    parent1 = list(parent1)
    parent2 = list(parent2)
    child = parent1[:split] + parent2[split:]
    child = np.asarray(child)
    return child
 #again, this heavily uses notes from our class   
def Mutate(child):
        # which gene to mutate?
    gene = np.random.randint(low=0, high=len(target))
    child[gene] = (np.random.randint(217))/2
    return child
#this is the main function from which we run the whole process     
def GeneticAlgorithm():
    #this is initialized to a random population first
    population = SuggestedInitialPopulation()   
    n_iterations = 2000
    p_mutation = 0.22
    fitness_goal = 0.12
    
    for k in range(n_iterations):
        new_generation = []
        print('Gen: ', k)
        
        list_of_p = CalculateProbability(population)
        for n in range(len(population)):
            
            parents = np.random.choice(range(0,10), size=2, p=list_of_p, replace=False)
            parent1, parent2 = population[parents[0]], population[parents[1]]
            child = Reproduce(parent1, parent2)
            l_mutate = np.random.choice([True, False], p=[p_mutation, 1-p_mutation])
            if l_mutate:
                child = Mutate(child)
            new_generation.append(child)
        population = new_generation
        
        performance = [recipe_success(state) for state in population]
        best_member = population[performance.index(max(performance))]
        print('Performance = ', max(performance))
        print('Best of gen=', best_member)
        

        
        if recipe_success(best_member) >= fitness_goal:
            print(best_member)
            return best_member
        
        
        
    
    
    
        
GeneticAlgorithm()

9
9
9
9
9
9
9
9
9
9
Gen:  0
Performance =  0.004640371229698376
Best of gen= [36  0  0  0  0 77  0  0  0]
Gen:  1
Performance =  0.007782101167315175
Best of gen= [36  0  0  0  0 77  0  0 87]
Gen:  2
Performance =  0.006872852233676976
Best of gen= [19  0  0  0  0 77  0  0 87]
Gen:  3
Performance =  0.006872852233676976
Best of gen= [19  0  0  0  0 77  0  0 87]
Gen:  4
Performance =  0.006872852233676976
Best of gen= [19  0  0  0  0 77  0  0 87]
Gen:  5
Performance =  0.006872852233676976
Best of gen= [19  0  0  0  0 77  0  0 87]
Gen:  6
Performance =  0.006389776357827476
Best of gen= [19  0 93  0 16 77  0  0 87]
Gen:  7
Performance =  0.006389776357827476
Best of gen= [19  0 93  0 16 77  0  0 87]
Gen:  8
Performance =  0.006153846153846154
Best of gen= [19 98 87  0  0 77  0  0 87]
Gen:  9
Performance =  0.0058309037900874635
Best of gen= [19 98  0  0  0 77  0  0 87]
Gen:  10
Performance =  0.006309148264984227
Best of gen= [19 83 98  0  0 77  0  0 87]
Gen:  11
Performance =  0.006309

Performance =  0.013793103448275862
Best of gen= [ 19  12  58   0   0 100   0   0  87]
Gen:  100
Performance =  0.01652892561983471
Best of gen= [ 19  48  58   0   0 100   0   0  87]
Gen:  101
Performance =  0.01652892561983471
Best of gen= [ 19  48  58   0   0 100   0   0  87]
Gen:  102
Performance =  0.01652892561983471
Best of gen= [ 19  48  58   0   0 100   0   0  87]
Gen:  103
Performance =  0.017094017094017096
Best of gen= [47 48 58  0  0 96  0  0 87]
Gen:  104
Performance =  0.017699115044247787
Best of gen= [ 47  20  58   0   0 100   0   0  89]
Gen:  105
Performance =  0.01834862385321101
Best of gen= [ 47  48  58   0   0 100   0   0  87]
Gen:  106
Performance =  0.01834862385321101
Best of gen= [ 47  48  58   0   0 100   0   0  87]
Gen:  107
Performance =  0.01834862385321101
Best of gen= [ 47  48  58   0   0 100   0   0  87]
Gen:  108
Performance =  0.01834862385321101
Best of gen= [ 47  48  58   0   0 100   0   0  87]
Gen:  109
Performance =  0.01834862385321101
Best of gen

Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  240
Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  241
Performance =  0.03508771929824561
Best of gen= [ 39  41  48   0   0 100   0   0 104]
Gen:  242
Performance =  0.03508771929824561
Best of gen= [ 39  41  48   0   0 100   0   0 104]
Gen:  243
Performance =  0.025974025974025976
Best of gen= [ 39  27  54   0   0 100   0   0 104]
Gen:  244
Performance =  0.025974025974025976
Best of gen= [ 39  27  54   0   0 100   0   0 104]
Gen:  245
Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  246
Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  247
Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  248
Performance =  0.028985507246376812
Best of gen= [ 39  41  54   0   0 100   0   0 104]
Gen:  249
Performance =  0.028985507246376

Performance =  0.024691358024691357
Best of gen= [ 39  41  46   0   0  88   0   0 102]
Gen:  366
Performance =  0.024691358024691357
Best of gen= [ 39  41  46   0   0  88   0   0 102]
Gen:  367
Performance =  0.024691358024691357
Best of gen= [ 39  41  46   0   0  88   0   0 102]
Gen:  368
Performance =  0.02247191011235955
Best of gen= [ 39  45  46   0   0  88   0   0 102]
Gen:  369
Performance =  0.020202020202020204
Best of gen= [ 39  41  46   0   0  88  11   0 102]
Gen:  370
Performance =  0.020202020202020204
Best of gen= [ 39  41  46   0   0  88  11   0 102]
Gen:  371
Performance =  0.020202020202020204
Best of gen= [39 41 46  0  0 88 11  0 90]
Gen:  372
Performance =  0.020202020202020204
Best of gen= [ 39  41  46   0   0  88  11   0 102]
Gen:  373
Performance =  0.03278688524590164
Best of gen= [ 39  41  46   0   0 107  11   0 102]
Gen:  374
Performance =  0.03278688524590164
Best of gen= [ 39  41  46   0   0 107  11   0 102]
Gen:  375
Performance =  0.03278688524590164
Best of

Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  453
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  454
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  455
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  456
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  457
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  458
Performance =  0.024096385542168676
Best of gen= [ 39  32  46   0   0  88   4   1 102]
Gen:  459
Performance =  0.023529411764705882
Best of gen= [ 39  32  46   0   0  88   5   1 102]
Gen:  460
Performance =  0.020618556701030927
Best of gen= [ 39  43  43   0   0  88   5   1 102]
Gen:  461
Performance =  0.020618556701030927
Best of gen= [ 39  43  43   0   0  88   5   1 102]
Gen:  462
Performance =  0.0206185567010

...

Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1400
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1401
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1402
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1403
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1404
Performance =  0.07407407407407407
Best of gen= [ 39  38  48   2   1 105   2   2  97]
Gen:  1405
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1406
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1407
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1408
Performance =  0.05128205128205128
Best of gen= [39 38 48  2  1 99  2  2 97]
Gen:  1409
Performance =  0.046511627906976744
Best of gen= [39 38 50  2  1 99  2  2 97]
Gen:  1410
Performance =  0.04651

Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1546
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1547
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1548
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1549
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1550
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1551
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1552
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1553
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1554
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1555
Performance =  0.04081632653061224
Best of gen= [39 37 49  2  1 99  2  7 97]
Gen:  1556
Performance =  0.042553191489361

Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1662
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1663
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1664
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1665
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1666
Performance =  0.0273972602739726
Best of gen= [34 37 60  2  3 99  2  7 97]
Gen:  1667
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1668
Performance =  0.03076923076923077
Best of gen= [ 34  37  60   2   3 103   2   7  97]
Gen:  1669
Performance =  0.0273972602739726
Best of gen= [34 37 60  2  3 99  2  7 97]
Gen:  1670
Performance =  0.0273972602739726
Best of gen= [34 37 60  2  3 99  2  7 97]
Gen:  1671
Performance =  0.028985507246376812
Best of gen= [ 34  37  3

Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1786
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1787
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1788
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1789
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1790
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1791
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1792
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1793
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1794
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1795
Performance =  0.04081632653061224
Best of gen= [34 37 48  2  3 99  2  7 97]
Gen:  1796
Performance =  0.040816326530612

Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1917
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1918
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1919
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1920
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1921
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1922
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1923
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1924
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1925
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1926
Performance =  0.020202020202020204
Best of gen= [33 29 48  1  3 83  5  7 97]
Gen:  1927
Performance =  0.0202

### (3c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

Responding to new prompts from Piazza:
1. I maximized the function with min = 0 and max = 1 (although the max I hit was 0.2). 
2. By changing p_permutation and n_iterations, I was able to match a maximum of 4 values (at 0.22 probability and 2000 iterations). 
3. I did a maximum of 2000 iterations. The effect seems to decline after a 1000 iterations. Barely has any effect after that.